In [11]:
%load_ext autoreload
%autoreload 2
import concurrent.futures
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize

notebook_name = "gridrad_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-21T03:30:00"
event_start = "2010-01-20"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / "runs/gridrad_demo"

In [13]:
# if output_parent.exists():
    # shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options, event_start=event_start
)

lon_range = [-102, -89]
lat_range = [27, 39]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.mcs(
    dataset="gridrad",
    global_flow_margin=70,
    unique_global_flow=False,
)

option.check_options(track_options)
option.save_track_options(track_options, options_directory=options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options = None

2024-10-09 17:38:17,899 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/data.yml
2024-10-09 17:38:17,951 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-09 17:38:17,952 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/grid.yml
2024-10-09 17:38:17,962 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/options/track.yml


In [4]:
times = data.utils.generate_times(data_options["gridrad"])
track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_parent
)

2024-10-09 17:23:44,812 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/gridrad_demo.
2024-10-09 17:23:44,814 - thor.track - INFO - Beginning simultaneous tracking.


2024-10-09 17:23:45,007 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-10-09 17:23:45,009 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-10-09 17:23:45,009 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-10-09 17:23:45,010 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T18:00:00.000000000.
2024-10-09 17:23:45,128 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-09 17:23:45,757 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-09 17:23:45,812 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-09 17:23:46,040 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-09 17:23:46,260 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-09 17:23:46,538 - thor.data.gridrad - DEBUG - Got domain mask for gridrad.
2024-10-09 17:23:47,127 - thor.data.gr

KeyboardInterrupt: 

In [5]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = []
    for i, time_interval in enumerate(intervals):
        args = [i, time_interval, data_options.copy(), grid_options.copy()]
        args += [track_options.copy(), visualize_options.copy()] 
        args += [output_parent, "gridrad"]
        futures.append(executor.submit(parallel.track_interval, *args))
    parallel.check_futures(futures)

2024-10-09 17:23:53,048 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/data.yml
2024-10-09 17:23:53,049 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_1/options/data.yml
2024-10-09 17:23:53,049 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_2/options/data.yml
2024-10-09 17:23:53,049 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_3/options/data.yml
2024-10-09 17:23:53,052 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_4/options/data.yml
2024-10-09 17:23:53,069 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/grid.yml
2024-10-09 17:23:53,073 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/gridrad_demo/interval_2/options/grid.yml
2024-10-09 17:23:53,073 - thor.option - DEBUG - Saving 

In [8]:
parallel.stitch_run(output_parent, intervals, cleanup=False)

2024-10-09 17:33:21,252 - thor.parallel - INFO - Stitching all attribute, mask and record files.


ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/object/apply:numpy._core.multiarray._reconstruct'
  in "/home/ewan/THOR_output/runs/gridrad_demo/interval_0/options/grid.yml", line 2, column 11

In [6]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent)

2024-10-09 14:09:04,523 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.csv
2024-10-09 14:09:04,526 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/velocities.yml
2024-10-09 14:09:04,651 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.csv
2024-10-09 14:09:04,654 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/quality.yml
2024-10-09 14:09:04,702 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.csv
2024-10-09 14:09:04,704 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/gridrad_demo/analysis/classification.yml


stratiform_offset inflow  \
time                universal_id                            
2010-01-20 18:00:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:10:00 1                      leading  right   
                    2                      leading  right   
2010-01-20 18:20:00 1                      leading  right   
...                                            ...    ...   
2010-01-21 03:10:00 8                        right  front   
                    20                       right  front   
                    25                    trailing  front   
2010-01-21 03:20:00 8                      leading  front   
                    20                       right  front   

                                 relative_stratiform_offset  \
time                universal_id                              
2010-01-20 18:00:00 1                                  left   
                    2                                  left   
2010-01-20 18:10:00 1                                  left   
                    2                                  left   
2010-01-20 18:20:00 1                                  left   
...                                                     ...   
2010-01-21 03:10:00 8                               leading   
                    20                                right   
                    25                             trailing   
2010-01-21 03:20:00 8                               leading   
                    20                                right   

                                                 tilt          propagation  
time                universal_id                                            
2010-01-20 18:00:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:10:00 1                      down-shear           down-shear  
                    2                      down-shear  shear-perpendicular  
2010-01-20 18:20:00 1                      down-shear           down-shear  
...                                               ...                  ...  
2010-01-21 03:10:00 8                      down-shear           down-shear  
                    20            shear-perpendicular           down-shear  
                    25                       up-shear           down-shear  
2010-01-21 03:20:00 8                      down-shear           down-shear  
                    20            shear-perpendicular           down-shear  

[248 rows x 5 columns]

In [3]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2010-01-20T18:00")
end_time = np.datetime64(np.datetime64("2010-01-21T03:30"))
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 5400, "by_date": False}
visualize.attribute.mcs_series(*args, **args_dict)

2024-10-09 14:48:11,567 - thor.data.gridrad - DEBUG - Converting GridRad dataset at time 2010-01-20T18:00:00.000000000.
2024-10-09 14:48:11,665 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-10-09 14:48:12,257 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-10-09 14:48:12,307 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-09 14:48:12,543 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-10-09 14:48:12,764 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-10-09 14:48:13,094 - thor.data.gridrad - DEBUG - Got domain mask for gridrad.
2024-10-09 14:48:13,199 - thor.visualize.attribute - DEBUG - Getting grid from dataset at time 2010-01-20T18:00:00.000000000.
2024-10-09 14:48:13,201 - thor.data.gridrad - DEBUG - Got grid from dataset at time 2010-01-20T18:00:00.000000000.
2024-10-09 14:48:13,202 - thor.visualize.attribute - DEBUG - Rebuilding processed grid for time